In [ ]:
import pandas as pd
import json
import numpy as np
import time
import re
from openai import OpenAI
from scipy.optimize import minimize
import os

## generate the initial user's risk preference conversation

In [ ]:
api_key="your GPT API KEY"

In [ ]:
client = OpenAI(api_key=api_key)

In [ ]:
SURVEY_CSV = 'survey_data/IR Stock Questionaires (Responses).xlsx'
USER_JSON = 'summary_data/'
STOCK_RAW_JSON = 'multi_assets_20251017.json'
user_conversation_path = 'user_conversation/'

In [ ]:
def load_and_preprocess():
    survey_df = pd.read_excel(SURVEY_CSV)
    with open(STOCK_RAW_JSON, 'r') as fb:
        raw_data = json.load(fb)
    return survey_df, raw_data

In [ ]:
def load_user_annotation(user_name):
    user_json_list = os.listdir(USER_JSON)
    user_json = None
    for file_name in user_json_list:
        if isinstance(file_name, str) and user_name in file_name:
            user_json=file_name
    if user_json:
        print(f"get user {user_name}'s run data")
        with open(os.path.join(USER_JSON, user_json), 'r') as fb:
            run_data = json.load(fb)
        return run_data
    else:
        print(f"No any user's name is {user_name}, please check it again!")
        return None

In [ ]:
survey_df, raw_data = load_and_preprocess() 

In [ ]:
survey_df

In [ ]:
column_mapping = {
    'Income': survey_df.columns[1],
    'Liquidity': survey_df.columns[2],
    'Assets': survey_df.columns[3],
    'Years_Exp': survey_df.columns[4],
    'Goals': survey_df.columns[5],
    'Instruments': survey_df.columns[6],
    'Risk_Attitude': survey_df.columns[7],
    'Gender': survey_df.columns[8],
    'Age': survey_df.columns[9],
    'Occupation': survey_df.columns[10],
    'Life_Stage': survey_df.columns[11],
    'Expenditure': survey_df.columns[12],
    'Decline_Reaction': survey_df.columns[13],
    'Social_FOMO': survey_df.columns[14],
    'Literacy': survey_df.columns[15],
    'State': survey_df.columns[16],
    'Emergency_Fund': survey_df.columns[17],
    'Liabilities': survey_df.columns[18]
}

In [ ]:
def calculate_max_drawdown(prices):
    """Compute maximum drawdown (peak-to-trough) within a 7-day window."""
    prices = np.array(prices)
    if len(prices) < 2:
        return 0.0
    running_max = np.maximum.accumulate(prices)
    drawdowns = (running_max - prices) / running_max
    return np.max(drawdowns)

In [ ]:
def get_market_based_ranks(snapshot):
    """Generate global Return and Volatility rankings based on a market snapshot."""
    
    # 7-day return ranking (high to low)
    ret_rank = sorted(
        snapshot.keys(),
        key=lambda x: float(snapshot[x]['7d_ret'].strip('%')),
        reverse=True
    )
    
    # Volatility ranking (low to high; lower is safer)
    vol_rank = sorted(
        snapshot.keys(),
        key=lambda x: float(snapshot[x]['vol'])
    )
    
    return ret_rank, vol_rank

In [ ]:
def generate_detailed_onboarding_prompt(row, column_mapping):
    # Construct a "Question: Answer" text block
    survey_details = ""
    for key, col_name in column_mapping.items():
        question_text = col_name  # Original survey question text
        answer_text = row[col_name]  # User's corresponding answer
        survey_details += f"Question: {question_text}\nAnswer: {answer_text}\n\n"

    # Build the enhanced prompt
    prompt = f"""
Act as a Senior Financial Psychologist and Scene Director. Your goal is to simulate an 'Onboarding Interview' between a Professional Advisor and a Client.

### [CLIENT'S TRUTH: SURVEY RESPONSES]
This is the raw data collected from the client's initial survey. Use these details to anchor the client's personality, vocabulary, and financial anxiety.

{survey_details}

### [SIMULATION TASK]
Conduct a high-fidelity, 4-turn dialogue (Advisor-Client pairs). 
1. **Turn 1 (Social & Background):** The Advisor introduces themselves and asks about the client's professional life and general situation. The Client responds, revealing their life stage and current mood.
2. **Turn 2 (Financial Foundation):** The Advisor probes into assets, liabilities, and income stability. The Client discusses their financial 'safety net' (or lack thereof), reflecting their real-world constraints (like debt or low liquidity).
3. **Turn 3 (Ambitions & Conflicts):** The Advisor asks about investment goals and experience. If the client is a "Beginner" but wants "Active Trading," the Advisor must gently point out this risk, and the Client should explain their internal motivation (e.g., 'need to catch up' or 'FOMO').
4. **Turn 4 (The Stress Test):** The Advisor simulates a market crash scenario based on the client's risk profile. The Client reveals their deepest fears and instinctive reactions to loss.

### [SCENE CONSTRAINTS]
- **No Scripted Language:** Avoid generic AI phrases. The Client should use first-person language ("I feel...", "My debt is...").
- **Literacy Matching:** If the survey says the client is a 'Beginner', they must NOT use professional jargon. They should sound like a layperson.
- **Hidden Signals:** Use the 'Emergency Fund' and 'Liabilities' data to color the client's level of confidence or desperation.
- **Consistency:** Ensure the client's tone remains consistent with their 'Risk Attitude' and 'Decline Reaction'.

### [OUTPUT FORMAT]
Return ONLY a valid JSON array of objects, like this:
[
  {{"role": "advisor", "content": "..."}},
  {{"role": "user", "content": "..."}}
]
"""
    return prompt

In [ ]:
def build_market_master_dict_enhanced(raw_data):
    """
    Enhanced version: provides not only the conversational snapshot,
    but also raw features for utility computation.
    """
    tickers = [key.replace('_DAILY_LAST30D', '') for key in raw_data.keys()]
    first_key = list(raw_data.keys())[0]
    dates = [entry['date'] for entry in raw_data[first_key]]
    daily_closes = {
        k.replace('_DAILY_LAST30D', ''): [e['close'] for e in raw_data[k]]
        for k in raw_data.keys()
    }
    
    market_master = {}
    for step in range(1, 24):
        window_start, window_end = step - 1, step + 5
        decision_date = dates[window_end + 1]
        snapshot = {}
        
        for ticker in tickers:
            prices = daily_closes[ticker][window_start: window_end + 1]
            daily_rets = np.diff(prices) / prices[:-1]
            
            # --- Core metric computation ---
            mu = np.mean(daily_rets)       # Expected return
            var = np.var(daily_rets)       # Return variance
            mdd = calculate_max_drawdown(prices)  # Maximum drawdown
            
            snapshot[ticker] = {
                "mu": mu,
                "var": var,
                "mdd": mdd,  # Raw floating-point values used for utility formula
                "7d_ret": f"{(prices[-1] / prices[0] - 1):.2%}",
                "vol": f"{np.std(daily_rets):.4f}",
                "current_price": f"${prices[-1]:.2f}",
                "price_trend": [f"${p:.2f}" for p in prices],
            }
        market_master[f"step_{step}"] = {
            "date": decision_date,
            "snapshot": snapshot,
        }
    return market_master, tickers

## Phase 1 offline optimization

In [ ]:
# Traditional optimization
def calibrate_user_parameters(run_data, market_master, tickers):
    """
    Phase 1: Inverse Optimization
    Estimate the user's Financial DNA parameters via Maximum Likelihood Estimation (MLE)
    """
    ticker_to_idx = {t: i for i, t in enumerate(tickers)}
    trajectory = []

    # 1. Prepare training data: align 23-step user choices with corresponding market states
    for s in run_data['selections']:
        step_idx = s['step']
        if step_idx > 23:
            continue  # Only process the first 23 steps
        
        snap = market_master[f"step_{step_idx}"]["snapshot"]
        mus = np.array([snap[t]['mu'] for t in tickers])
        vars_ = np.array([snap[t]['var'] for t in tickers])
        mdds = np.array([snap[t]['mdd'] for t in tickers])
        chosen_idx = ticker_to_idx[s['asset']]
        
        trajectory.append({
            'mus': mus,
            'vars': vars_,
            'dds': mdds,
            'chosen_idx': chosen_idx
        })

    # 2. Define the Negative Log-Likelihood (NLL) loss function
    def nll_loss(params):
        lam, gamma = params
        loss = 0
        for step in trajectory:
            # Utility formula: U = mu - lambda * var - gamma * mdd
            utilities = step['mus'] - lam * step['vars'] - gamma * step['dds']
            
            # Softmax probability
            max_u = np.max(utilities)
            exp_u = np.exp(utilities - max_u)
            probs = exp_u / np.sum(exp_u)
            
            loss -= np.log(probs[step['chosen_idx']] + 1e-10)
        return loss

    # 3. Solve the optimization problem
    res = minimize(
        nll_loss,
        x0=[5.0, 0.5],
        bounds=[(0, None), (0, None)],
        method='L-BFGS-B'
    )

    (lambda_i, gamma_i) = (
        (res.x[0], res.x[1]) if res.success else (5.0, 5.0)
    )

    hit_rate = evaluate_dna_hit_rate(lambda_i, gamma_i, trajectory)
    return lambda_i, gamma_i, hit_rate

In [ ]:
# Standardized optimization

def calibrate_user_dna_standardized(run_data, market_master, tickers):
    ticker_to_idx = {t: i for i, t in enumerate(tickers)}
    trajectory = []

    for s in run_data['selections']:
        step_idx = s['step']
        if step_idx > 23:
            continue
        
        snap = market_master[f"step_{step_idx}"]["snapshot"]
        
        # Extract raw feature vectors
        mus = np.array([snap[t]['mu'] for t in tickers])
        vars_ = np.array([snap[t]['var'] for t in tickers])
        mdds = np.array([snap[t]['mdd'] for t in tickers])
        
        # --- Core improvement: cross-sectional standardization (Z-score) ---
        # Ensures the three feature groups contribute equally in variance
        # before entering the Softmax
        def scale(x):
            std = np.std(x)
            return (x - np.mean(x)) / (std if std > 1e-9 else 1.0)

        trajectory.append({
            'mus': scale(mus),
            'vars': scale(vars_),
            'dds': scale(mdds),
            'chosen_idx': ticker_to_idx[s['asset']]
        })

    def nll_loss(params):
        lam, gamma = params
        loss = 0
        for step in trajectory:
            # Here lambda and gamma represent relative weight ratios
            # across standardized indicators
            utilities = step['mus'] - lam * step['vars'] - gamma * step['dds']
            
            # Numerical stability handling
            max_u = np.max(utilities)
            exp_u = np.exp(utilities - max_u)
            probs = exp_u / (np.sum(exp_u) + 1e-10)
            
            loss -= np.log(probs[step['chosen_idx']] + 1e-10)
        
        # L2 regularization to prevent parameter drift in extreme cases
        reg = 0.01 * (lam**2 + gamma**2)
        return loss + reg

    # Use more reasonable initialization and bounds
    res = minimize(
        nll_loss,
        x0=[1.0, 1.0],
        bounds=[(0, 20), (0, 20)],
        method='L-BFGS-B'
    )

    (lambda_i, gamma_i) = (
        (res.x[0], res.x[1]) if res.success else (1.0, 1.0)
    )

    hit_rate = evaluate_dna_hit_rate(lambda_i, gamma_i, trajectory)
    return lambda_i, gamma_i, hit_rate

In [ ]:
# Regularized optimization
def calibrate_user_dna_regularized(run_data, market_master, tickers):
    ticker_to_idx = {t: i for i, t in enumerate(tickers)}
    trajectory = []

    for s in run_data['selections']:
        step_idx = s['step']
        if step_idx > 23:
            continue
        snap = market_master[f"step_{step_idx}"]["snapshot"]
        trajectory.append({
            'mus': np.array([snap[t]['mu'] for t in tickers]),
            'vars': np.array([snap[t]['var'] for t in tickers]),
            'dds': np.array([snap[t]['mdd'] for t in tickers]),
            'chosen_idx': ticker_to_idx[s['asset']]
        })

    def nll_loss(params):
        lam, gamma = params
        loss = 0
        for step in trajectory:
            # Original utility formula
            utilities = step['mus'] - lam * step['vars'] - gamma * step['dds']
            max_u = np.max(utilities)
            exp_u = np.exp(utilities - max_u)
            probs = exp_u / (np.sum(exp_u) + 1e-10)
            loss -= np.log(probs[step['chosen_idx']] + 1e-10)
        
        # Core improvement: asymmetric regularization
        # Since var is very small (~10^-5), lambda requires weaker penalty
        # Since mdd is relatively larger (~10^-2), gamma requires stronger penalty
        reg = 0.001 * lam**2 + 0.1 * gamma**2 
        return loss + reg

    res = minimize(
        nll_loss,
        x0=[10.0, 10.0],
        bounds=[(0, 1000), (0, 200)],
        method='L-BFGS-B'
    )

    lambda_i, gamma_i = res.x[0], res.x[1]
    hit_rate = evaluate_dna_hit_rate(lambda_i, gamma_i, trajectory)
    return lambda_i, gamma_i, hit_rate

In [ ]:
def evaluate_dna_hit_rate(lambda_i, gamma_i, trajectory):
    """
    诊断工具：计算拟合出的 DNA 在 23 步中能多大程度预测用户的真实选择
    """
    hits = 0
    for step in trajectory:
        # 计算效用值
        u = step['mus'] - lambda_i * step['vars'] - gamma_i * step['dds']
        # 预测排名第一的索引
        pred_idx = np.argmax(u)
        if pred_idx == step['chosen_idx']:
            hits += 1
    return hits / len(trajectory)

# 使用示例
# hit_rate = evaluate_dna_hit_rate(res.x[0], res.x[1], trajectory)
# print(f"DNA Prediction Accuracy: {hit_rate:.2%}")

## phase 2 Real-time rankings

In [ ]:
def get_utility_ground_truth(snapshot, lam, gam):
    """
    Phase 2: Compute the ground-truth ranking for the current step
    based on calibrated parameters.
    """
    scores = {}
    for ticker, data in snapshot.items():
        # Utility formula: U = mu - lambda * var - gamma * mdd
        u = data['mu'] - lam * data['var'] - gam * data['mdd']
        scores[ticker] = u
    
    # Return ranking from highest to lowest utility
    return sorted(scores.keys(), key=lambda x: scores[x], reverse=True)

In [ ]:
# 1. Build market snapshots and feature libraries
market_master_dict, tickers = build_market_master_dict_enhanced(raw_data)

In [ ]:
# market_master_dict

In [ ]:
# tickers

In [ ]:
def generate_multi_advisor_step_prompt(step_idx, history, snapshot, chosen_asset, feedback, ranks, lambda_i, gamma_i):
    # Constructing market display text
    market_elements = []
    for tk, v in snapshot.items():
        trend_str = f"{v['price_trend'][0]} -> {v['price_trend'][-1]}"
        market_elements.append(
            f"- {tk}: 7D Ret: {v['7d_ret']}, Var: {v['var']:.6f}, "
            f"Max Drawdown: {v['mdd']:.2%}, Trend: ({trend_str})"
        )
    market_text = "\n".join(market_elements)
    
    # Dynamically construct Advisor 1's internal personality cues (without revealing them to the User).
    dna_insight = []
    if lambda_i > 0.1: dna_insight.append("User dislikes price volatility.")
    if gamma_i > 0.1: dna_insight.append("User is extremely averse to peak-to-trough drawdowns.")
    insight_text = " ".join(dna_insight)

    prompt = f"""
### [SYSTEM ROLE]
You are simulating Step {step_idx}/23 of a financial advisory session. 
Act as 3 specialized Advisors and a Client.

### [EXPERT PERSONAS]
1. **Advisor 1 (Rationalist)**: Lead professional. Your advice follows a Mean-Variance-Drawdown utility model. You recommend the most balanced choice for the user's specific DNA.
2. **Advisor 2 (Momentum Hunter)**: Trend-focused. You chase the highest 7-day average returns (μ).
3. **Advisor 3 (Safety Manager)**: Risk-averse. You prioritize capital preservation (min σ² and Drawdown).

### [STRICT COMMUNICATION CONSTRAINTS]
- **NO MATHEMATICAL LEAKAGE**: Never mention "Utility functions", "Lambda/Gamma", "Z-scores", or formulas like "U = μ - λσ²".
- **PROFESSIONAL TRANSLATION**: Convert data into advice. 
    - *Poor*: "Your Gamma is high, so I pick X."
    - *Better*: "Given your preference for avoiding sharp market dips, I recommend X for its superior downside protection."

### [CONTEXT]
- **Advisor 1's Internal Insight**: {insight_text}
- **Market Snapshot**:
{market_text}
- **Last Round Feedback**: {feedback}
- **History**: {json.dumps(history[-4:], indent=2)}

### [INTERNAL GROUND TRUTH (For Advisor 1-3 Reference)]
- Utility Rank (Ideal for this user): {ranks['utility']}
- Momentum Rank: {ranks['return']}
- Safety Rank: {ranks['volatility']}

### [TASK]
1. **Panel Discussion**: Each advisor gives a 1-2 sentence recommendation. 
   - **Advisor 1** must advocate for **{ranks['utility'][0]}**. Explain why it's the most rational choice without using math.
2. **The User Decision**: The client MUST finalize the choice as **{chosen_asset}**. 
   - If {chosen_asset} != Advisor 1's top pick, the user must provide a subjective reason (e.g., "I'm feeling aggressive today" or "I'm ignoring the risk Advisor 3 mentioned").

### [OUTPUT FORMAT - JSON]
Return ONLY a JSON object:
{{
  "advisor_1_msg": "...",
  "advisor_2_msg": "...",
  "advisor_3_msg": "...",
  "user_msg": "...",
  "advisor_rankings": {{
      "utility_expert": {json.dumps(ranks['utility'])},
      "momentum_expert": {json.dumps(ranks['return'])},
      "safety_expert": {json.dumps(ranks['volatility'])}
  }}
}}
"""
    return prompt

In [ ]:
def parse_json_safely(response_str):
    """
    Clean and safely parse a JSON string returned by an LLM.
    """
    # 1. Try direct parsing (if the response is already clean)
    try:
        return json.loads(response_str)
    except json.JSONDecodeError:
        pass

    # 2. Remove Markdown code block markers
    # Match ```json ... ``` or ``` ... ```
    clean_str = re.sub(r'```json\s*|```\s*', '', response_str).strip()
    
    # 3. If still failing, attempt to extract the first {...} or [...] block via regex
    try:
        match = re.search(r'(\{.*\}|\[.*\])', clean_str, re.DOTALL)
        if match:
            return json.loads(match.group(1))
    except (json.JSONDecodeError, AttributeError):
        pass

    # 4. Raise error for debugging
    print(f"Failed to parse JSON. Raw response was: {response_str}")
    raise ValueError("LLM response is not a valid JSON format.")

In [ ]:
def get_response(user_input):
    response = client.responses.create(
        model="gpt-5.2-2025-12-11",
        input=user_input,
        reasoning={
            "effort": "low"
        },
        text={
            "verbosity": "low"
        }
    )
    return response.output_text

In [ ]:
def get_ground_truth_ranks(snapshot, tickers, lambda_i, gamma_i):
    """
    Core transformation function:
    Convert raw market snapshot into three standardized ground-truth rankings.
    """
    # 1. Extract raw feature vectors
    mus_raw = np.array([snapshot[tk]['mu'] for tk in tickers])
    vars_raw = np.array([snapshot[tk]['var'] for tk in tickers])
    mdds_raw = np.array([snapshot[tk]['mdd'] for tk in tickers])

    # 2. Define internal cross-sectional standardization function
    # (must match calibration logic exactly)
    def cross_sectional_scale(x):
        std = np.std(x)
        if std < 1e-12:
            return x - np.mean(x)
        return (x - np.mean(x)) / std

    # Apply standardization
    mus_s = cross_sectional_scale(mus_raw)
    vars_s = cross_sectional_scale(vars_raw)
    mdds_s = cross_sectional_scale(mdds_raw)

    # 3. Compute utility scores and ranking
    # Utility function: U = mu_s - lambda * var_s - gamma * mdd_s
    utility_scores = {
        tickers[i]: mus_s[i] - lambda_i * vars_s[i] - gamma_i * mdds_s[i]
        for i in range(len(tickers))
    }
    util_rank = sorted(
        utility_scores.keys(),
        key=lambda x: utility_scores[x],
        reverse=True
    )

    # 4. Compute momentum ranking (Momentum: based only on return mu)
    momentum_rank = sorted(
        tickers,
        key=lambda x: snapshot[x]['mu'],
        reverse=True
    )

    # 5. Compute safety ranking (Safety: penalize volatility and drawdown)
    # For the current user, risk is the weighted sum of standardized risk terms
    safety_scores = {
        tickers[i]: lambda_i * vars_s[i] + gamma_i * mdds_s[i]
        for i in range(len(tickers))
    }
    vol_rank = sorted(
        safety_scores.keys(),
        key=lambda x: safety_scores[x]
    )  # Lower score indicates safer asset

    return {
        'utility': util_rank,
        'return': momentum_rank,
        'volatility': vol_rank
    }

In [ ]:
def get_conversation(survey_df, user_conversation_path, start_index=0):
    
    for index, row in survey_df.iterrows():
        if index < start_index:
            continue
        
        # Each user corresponds to one run_data 
        # (assuming the mapping has already been handled)
        user_name = row["Name"]
        user_id = f"User_{index}"

        run_data = load_user_annotation(user_name)
        if run_data:
            # --- PHASE 1: Inverse optimization calibration (obtain global parameters for this user) ---
            print(f"Calibrating Financial DNA for {user_id}...")
            lambda_i, gamma_i, hit_rate = calibrate_user_dna_standardized(
                run_data, market_master_dict, tickers
            )
            print(f"Result -> Lambda: {lambda_i:.4f}, Gamma: {gamma_i:.4f}, Hit Rate: {hit_rate: .2%}")

            # 1. Onboarding (keep unchanged)
            onboarding_prompt = generate_detailed_onboarding_prompt(row, column_mapping)
            initial_response = get_response(onboarding_prompt)
            history = parse_json_safely(initial_response)
            
            history.append({
                "role": "system", 
                "content": "The onboarding is complete. A panel of experts has joined."
            })
            
            final_output = []
            
            # 2. Decision loop (Step 1–23)
            for t in range(1, 24):
                print(f"conversation for step {t}......")
                step_market_info = market_master_dict[f"step_{t}"]
                snapshot = step_market_info["snapshot"]
                
                # --- Call core function to obtain rankings ---
                ranks = get_ground_truth_ranks(snapshot, tickers, lambda_i, gamma_i)
                
                # Retrieve ground-truth selection and feedback
                selection = next(s for s in run_data['selections'] if s['step'] == t)
                chosen_asset = selection['asset']
                
                # Build feedback text
                feedback = (
                    f"Last return: {selection['ret']:.2%}" 
                    if t > 1 else "First step."
                )
            
                # Generate prompt and call LLM
                prompt = generate_multi_advisor_step_prompt(
                    t, history, snapshot, chosen_asset, feedback, ranks, lambda_i, gamma_i
                )
                raw_response = get_response(prompt)
                
                try:
                    res = parse_json_safely(raw_response)
                    
                    step_entry = {
                        "step": t,
                        "date": step_market_info["date"],
                        "calibrated_dna": {"lambda": lambda_i, "gamma": gamma_i},
                        "market_snapshot": snapshot,
                        "advisor_panel": {
                            "utility_advisor": {
                                "msg": res['advisor_1_msg'], 
                                "rank": ranks['utility']
                            },
                            "momentum_advisor": {
                                "msg": res['advisor_2_msg'], 
                                "rank": ranks['return']
                            },
                            "safety_advisor": {
                                "msg": res['advisor_3_msg'], 
                                "rank": ranks['volatility']
                            }
                        },
                        "user_turn": {
                            "msg": res['user_msg'], 
                            "choice": chosen_asset
                        }
                    }
                    
                    # Update conversation history
                    history_text = (
                        f"Panel Discussion - Advisor1: {res['advisor_1_msg']} "
                        f"Advisor2: {res['advisor_2_msg']} "
                        f"Advisor3: {res['advisor_3_msg']}"
                    )
                    history.append({"role": "advisor", "content": history_text})
                    history.append({"role": "user", "content": res['user_msg']})
                    
                    final_output.append(step_entry)
                    
                except Exception as e:
                    print(f"Error at step {t}: {e}")
                    continue
            
            with open(
                os.path.join(user_conversation_path, f"{user_id}_conversation_history.json"), 
                "w"
            ) as f:
                json.dump(history, f, indent=4)

            with open(
                os.path.join(user_conversation_path, f"{user_id}_conversation_select.json"), 
                "w"
            ) as fb:
                json.dump(final_output, fb, indent=4)

In [ ]:
get_conversation(survey_df,user_conversation_path, start_index=9)